# CS229 :Problem Set 4
## Problem 4: 独立成分分析


## 符号定义
+ $X$表示混淆过的声音数据矩阵
+ $S$表示分离后的声音数据矩阵
+ $x^i$表示第i个混淆声音样本
+ $s^i$表示第i个原声音样本
+ $A$代表混淆矩阵
+ $W$代表逆混淆矩阵

## ICA算法流程简介

1. 假设 $x^i=As^i $
2. 原声为 $s^i = Wx^i$
3. 在已知原生S的概率密度分布$p_S(s)$的情况下，求出混淆声X的概率密度
$$F_X(x)=F(X \ge x) = F(AS \ge x) = F(S \ge Wx)  $$
$$p_X(x)=F_X(x)'=F(S \ge Wx)' = p_s(Wx)|W| $$
4. 现在选取sigmoid函数来成为$p_s(s)$的分布函数，即$g(s)'=p_s(s)$。*因为sigmoid函数满足分布函数的定义，且其通常是一个好的选择*
5. 目标函数：
![](https://yunlongs-1253041399.cos.ap-chengdu.myqcloud.com/image/Stanford/lecture-15-13.png)
6.随机梯度的更新公式：
![](https://yunlongs-1253041399.cos.ap-chengdu.myqcloud.com/image/Stanford/lecture-15-12.jpg)

In [1]:
### 安装sounddevic库：
### On Mac:
###     1. portaudio: On Mac: brew install portaudio
###     2. sounddevice: pip install sounddevice
###
### On windows:
###      pip install pyaudio sounddevice
###

下面这部分不属于算法的部分，是属于音频调制的部分

In [2]:
import sounddevice as sd
import numpy as np
Fs = 11025


def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

def play(vec):
    sd.play(vec, Fs, blocking=True)

逆混淆函数，学习出参数W

In [3]:
def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]  ## 逐渐变小学习速率
    print('Separating tracks ...')
    ######## Your code here ##########
    for alpha in anneal:  ## 使用每一个因子进行一次完整的训练
        order = np.arange(M)
        np.random.shuffle(order)   ## 将m个样本的更新顺序打乱
        x = X[order, :] 
        for i in range(M):
            a = x[i, :]
            g =  1/(1+np.exp(-(W.dot(x[i,:].reshape(N,1)))))    ## sigmoid
            W += alpha * (np.dot((1-2*g.reshape(N,1)),x[i,:].reshape(1,N)) + np.linalg.inv(W.T))
    return W

恢复出每个原声S

In [4]:
def unmix(X, W):
    S = np.zeros(X.shape)

    ######### Your code here ##########
    S = np.dot(X,W.T)
    ##################################
    return S


In [5]:
def main():
    X = normalize(load_data())

    for i in range(X.shape[1]):
        print('Playing mixed track %d' % i)
        #play(X[:, i])

    W = unmixer(X)
    S = normalize(unmix(X, W))

    for i in range(S.shape[1]):
        print('Playing separated track %d' % i)
        play(S[:, i])

if __name__ == '__main__':
    main()

Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4
Separating tracks ...
Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4
